In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

In [2]:
data_df = pd.read_csv("titanic/train.csv")

In [3]:
data_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
data_df.shape

(891, 12)

In [5]:
data_df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [6]:
data_df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis = 1, inplace = True)

In [7]:
data_df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [8]:
data_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


# Split the data into train and test set

In [9]:
X_train, X_test, y_train, y_test = train_test_split(data_df.drop('Survived', axis = 1), data_df['Survived'], test_size = 0.2)

In [10]:
X_train.shape, X_test.shape

((712, 7), (179, 7))

# Let's create column transformers

In [11]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
115,3,male,21.0,0,0,7.925,S
417,2,female,18.0,0,2,13.000,S
446,2,female,13.0,0,1,19.500,S
249,2,male,54.0,1,0,26.000,S
672,2,male,70.0,0,0,10.500,S


In [18]:
# 1) Handle the missing values

ctrf_impute = ColumnTransformer([
                                    ("impute_age", SimpleImputer(), [2]),
                                    ("impute_embarked", SimpleImputer(strategy="most_frequent"), [6])
                                ], remainder = 'passthrough')

In [19]:
transform_pipe = Pipeline([
                            ("ctrf_impute", ctrf_impute)
                        ])

In [20]:
transform_pipe.fit_transform(X_train)

array([[21.0, 'S', 3, ..., 0, 0, 7.925],
       [18.0, 'S', 2, ..., 0, 2, 13.0],
       [13.0, 'S', 2, ..., 0, 1, 19.5],
       ...,
       [51.0, 'S', 1, ..., 1, 0, 77.9583],
       [33.0, 'S', 2, ..., 1, 2, 27.75],
       [3.0, 'S', 3, ..., 4, 2, 31.3875]], dtype=object)

# The column order is not preserved after applying column transformer and also, we have to set remainder to passthrough, otherwise, we will loose other columns (features)

# How to preserve column orders while using columntransformers

In [21]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
115,3,male,21.0,0,0,7.925,S
417,2,female,18.0,0,2,13.000,S
446,2,female,13.0,0,1,19.500,S
249,2,male,54.0,1,0,26.000,S
672,2,male,70.0,0,0,10.500,S


In [22]:
# handle missing values:
ctrf_impute = ColumnTransformer([  ("pclass_sex", "passthrough", [0,1]),
                                   ("impute_age", SimpleImputer(), [2]),
                                   ("sibsp_parch_fare", "passthrough", [3,4,5]),
                                   ("impute embarked", SimpleImputer(strategy = 'most_frequent'), [6])
                                ], verbose = True, remainder='passthrough')

In [23]:
# Handle categorial columns
ctrf_categorical = ColumnTransformer([  ("pclass", "passthrough", [0]),
                                        ("one_hot_encoding_sex", OneHotEncoder(sparse=False, handle_unknown='ignore'), [1]),
                                        ("age_sibsip_parch_fare", "passthrough", [2,3,4,5]),
                                        ("one_hot_encoding_embarked", OneHotEncoder(sparse=False, handle_unknown='ignore'), [6])
                                     ], remainder='passthrough')

In [24]:
# Scaling
ctrf_scaling = ColumnTransformer([
                                    ("standard_scalar", StandardScaler(), [3,6])
                                 ], remainder='passthrough')

In [26]:
clf = RandomForestClassifier()

# Lets create a pipeline

In [27]:
model = Pipeline([
                    ("handle missing values", ctrf_impute),
                    ("handle categorical columns", ctrf_categorical),
                    ("scaling", ctrf_scaling),
                    ("train", clf)
                ])

In [28]:
model

Pipeline(steps=[('handle missing values',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('pclass_sex', 'passthrough',
                                                  [0, 1]),
                                                 ('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('sibsp_parch_fare',
                                                  'passthrough', [3, 4, 5]),
                                                 ('impute embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])],
                                   verbose=True)),
                ('handle categorical columns',
                 ColumnTransformer(remainder='passth...
                                                 ('one_hot_encoding_sex',
                               

In [29]:
from sklearn import set_config

In [30]:
set_config(display = 'diagram')

In [31]:
model

Pipeline(steps=[('handle missing values',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('pclass_sex', 'passthrough',
                                                  [0, 1]),
                                                 ('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('sibsp_parch_fare',
                                                  'passthrough', [3, 4, 5]),
                                                 ('impute embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])],
                                   verbose=True)),
                ('handle categorical columns',
                 ColumnTransformer(remainder='passth...
                                                 ('one_hot_encoding_sex',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1]),
                                                 ('age_sibsip_parch_fare',
                                                  'passthrough', [2, 3, 4, 5]),
                                                 ('one_hot_encoding_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [6])])),
                ('scaling',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('standard_scalar',
                                                  StandardScaler(), [3, 6])])),
                ('train', RandomForestClassifier())])

In [32]:
model.fit(X_train, y_train)

[ColumnTransformer] .... (1 of 4) Processing pclass_sex, total=   0.0s
[ColumnTransformer] .... (2 of 4) Processing impute_age, total=   0.0s
[ColumnTransformer]  (3 of 4) Processing sibsp_parch_fare, total=   0.0s
[ColumnTransformer]  (4 of 4) Processing impute embarked, total=   0.0s


Pipeline(steps=[('handle missing values',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('pclass_sex', 'passthrough',
                                                  [0, 1]),
                                                 ('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('sibsp_parch_fare',
                                                  'passthrough', [3, 4, 5]),
                                                 ('impute embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])],
                                   verbose=True)),
                ('handle categorical columns',
                 ColumnTransformer(remainder='passth...
                                                 ('one_hot_encoding_sex',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1]),
                                                 ('age_sibsip_parch_fare',
                                                  'passthrough', [2, 3, 4, 5]),
                                                 ('one_hot_encoding_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [6])])),
                ('scaling',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('standard_scalar',
                                                  StandardScaler(), [3, 6])])),
                ('train', RandomForestClassifier())])

In [33]:
model.score(X_train, y_train)

0.9817415730337079

In [34]:
predictions = model.predict(X_test)

In [35]:
from sklearn.metrics import accuracy_score

In [36]:
accuracy_score(y_test, predictions)

0.7877094972067039